In [2]:
# good tutorial for elasticsearch python http://nitin-panwar.github.io/Elasticsearch-tutorial-for-beginners-using-Python/
#https://elasticsearch-py.readthedocs.io/en/master/
#http://snowsyn.net/2016/10/25/having-fun-with-elasticsearch-and-python/
#https://marcobonzanini.com/2015/02/02/how-to-query-elasticsearch-with-python/
#https://www.elastic.co/guide/en/elasticsearch/guide/current/routing-value.html for the report how distributed indexing works
#http://xpo6.com/using-elasticsearch-analyzer-remove-stop-words-from-text/
#https://www.elastic.co/guide/en/elasticsearch/guide/current/data-in-data-out.html  every field has a dedicated inverted index for fast retrieval.
#https://dzone.com/articles/23-useful-elasticsearch-example-queries

from elasticsearch import Elasticsearch 
# Connect to the elastic cluster
es=Elasticsearch([{'host':'localhost','port':9200}])
es

<Elasticsearch([{'host': 'localhost', 'port': 9200}])>

In [3]:
e1={
    "first_name":"nitin",
    "last_name":"panwar",
    "age": 27,
    "about": "Love to play cricket",
    "interests": ['sports','music'],
}
print(e1)

{'first_name': 'nitin', 'last_name': 'panwar', 'age': 27, 'about': 'Love to play cricket', 'interests': ['sports', 'music']}


In [15]:
res = es.index(index='megacorp',doc_type='employee',id=1,body=e1)

In [8]:
# Let's insert some more documents
e2={
    "first_name" :  "Jane",
    "last_name" :   "Smith",
    "age" :         32,
    "about" :       "I like to collect rock albums",
    "interests":  [ "music" ]
}
e3={
    "first_name" :  "Douglas",
    "last_name" :   "Fir",
    "age" :         35,
    "about":        "I like to build cabinets",
    "interests":  [ "forestry" ]
}

In [19]:
res=es.index(index='megacorp',doc_type='employee',id=2,body=e2)
res=es.index(index='megacorp',doc_type='employee',id=3,body=e3)

In [21]:
res=es.get(index='megacorp',doc_type='employee',id=3)
print(res)

{'_index': 'megacorp', '_type': 'employee', '_id': '3', '_version': 1, '_seq_no': 0, '_primary_term': 1, 'found': True, '_source': {'first_name': 'Douglas', 'last_name': 'Fir', 'age': 35, 'about': 'I like to build cabinets', 'interests': ['forestry']}}


In [23]:
print(res['_source'])

{'first_name': 'Douglas', 'last_name': 'Fir', 'age': 35, 'about': 'I like to build cabinets', 'interests': ['forestry']}


In [84]:
#res=es.delete(index='megacorp',doc_type='employee',id=1)
#res=es.delete(index='megacorp',doc_type='employee',id=2)
#res=es.delete(index='megacorp',doc_type='employee',id=3)

#job = {
#    "title": "technology returner programme", "company": "nomura", "description": "nomura diversity critical success harness value perspectives employees different work family experience backgrounds bring firm past experience invaluable us nomura technology division pleased launching first returner programme looking individuals taken career break 18months much longer looking return workplace successful candidates receive significant training support help transition new role world technology fast moving excellent opportunities find fulfilling role also provides flexibility need roles available 4 5 day week basis nomura financial services group integrated global network spanning 30 countries head offices asia connecting markets east west service needs individuals institutions corporates governments four business divisions retail asset management wholesale global markets investment banking merchant banking offer technology returner programme based london office designed experienced professionals ready build previous skills experience want embrace new opportunities ahead paid programme commences april 29th finishes 19th july 2019. end 3-month programme participants invited apply number open roles offer number type roles available outlined programme \u2019 guarantee permanent role ensure developed valuable skills exposure wide-reaching influential network involvement high impact projects global financial services company looking returners join us previous experience business analysts project managers developers specific opportunities offer include strategic projects manager cto office software engineer business analyst team leader security consultant project manager hybrid project manager/it business analyst programme open women men career break least 18 months minimum number 5 years professional experience need able work city based london office duration programme eligible work uk april 2019 july 2019. opportunities available 4 5 days week basis pending preference supported goal support back workplace get taste career global investment bank support line manager coach buddy also employee networks join build network across firm prior 12 weeks provided pre-commencement coaching training day inclusivity one day induction organisation three personal coaching sessions delivered inclusivity \u2019 specialist coach relevant technical training role remuneration benefits opportunity salary aligned market rates also access programme nomura services emergency childcare adult dependent care fitness centre onsite restaurant onsite gp nurse dental practice apply help application process questions hesitate contact us submit c.v. cover letter online tuesday 5th march latest please ensure c.v. shows career break least 18 months immediately preceding date application programme help us get know better include cover letter 've taken break work include motivates want return please place cover letter document cv shortlisted inclusivity touch organise first interview please click 'apply button redirected website", "location": "london", "URL": "https://www.technojobs.co.uk/job/2620662/technology-returner-programme/"}, {"title": "mobile developer in test javascript appium mocha chai", "company": "client server", "description": "mobile developer test javascript appium mocha chai city based financial software house provide range web based trading single-dealer platforms seeking mobile developer test experience testing mobile front end applications six month contract city office mobile developer test responsible developing automated test cases test frameworks new functionality within jenkins environment include functional non-functional testing using variety open source tools libraries 'll collaborate ux designers product owners define implement unit acceptance integration performance tests relaxed city based offices table tennis football casual dress code flexible working requirements *experience testing mobile applications appium *experience javascript testing frameworks including mocha chai *experience working jenkins *experience software tester developer test good appreciation test automation *passionate code quality advocate clean code principles *excellent analysis problem solving skills meticulous attention detail *nice mobile testing experience react react native apply call confidential discussion developer test contract opportunity mobile developer test javascript appium mocha chai rate \u00a3400 \u00a3450 p/day location london city term 6 months start immediate asap", "location": "london", "URL": "https://www.technojobs.co.uk/job/2624214/mobile-developer-in-test-javascript-appium-mocha-chai/"
#    }

#res=es.index(index='technojobs',doc_type='jobs',id=1,body=job)
#print(res)
#es.indices.delete(index='job', ignore=[400, 404])

{'acknowledged': True}

In [85]:
es.indices.create(index='job', body={
   'settings' : {
         'index' : {
              'number_of_shards':5
         },
    "analysis": {
      "filter": {
        "filter_stemmer": {
          "type": "stemmer",
          "language": "english"
        },
            "stopwordEn": {
            "type":       "stop",
            "stopwords":  "_english_"
        }
      },
      "analyzer": {
        "tags_analyzer": {
          "type": "custom",
          "filter": [
            "standard",
            "lowercase",
            "filter_stemmer"
          ],
          "tokenizer": "standard"
        }
      }
    }
   }
})

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'job'}

In [86]:
import json
    
with open('all_jobs.json', 'r') as f:
    technojobs_dict = json.load(f)
    #count = 0
    for technojob in technojobs_dict:
        #count+=1
        my_id = technojob.pop('URL', None)
        es.index(index='job', doc_type='job', body=json.dumps(technojob))
        #if count >= 10:
        #    break
    
#for technojob in technojobs_dict:
#    print(technojob['title'])
    



In [87]:
def searchFunction(feature, search):
    res = es.search(index="job", doc_type="job", body={"query": {"match": {feature: search}}})
    print("%d documents found" % res['hits']['total'])
    for doc in res['hits']['hits']:
        print("%s) %s" % (doc['_id'], doc['_source'][feature]))
        
def searchFunctionAllFeatures(search):
    res = es.search(index="job", doc_type="job", body={"query": {"multi_match" : {
            "query" : search,
            "fields": ["title^3", "description","location","URL"]#makes the title three times more important
        }}})
    print("%d documents found" % res['hits']['total'])
    for doc in res['hits']['hits']:
        print("%s) %s" % (doc['_id'], doc['_source']))

In [93]:
searchFunction("title","data scientist")

139 documents found
6A2EFWkBP7Y1UR11_nWA) data scientist
5w2EFWkBP7Y1UR11-nQr) lead data scientist uk
og2FFWkBP7Y1UR11AXaX) data engineer data scientist
hQ2FFWkBP7Y1UR11BXeB) senior data scientist
SQ2EFWkBP7Y1UR115nCk) data scientist ai engineer
Vg2FFWkBP7Y1UR11AHZV) data scientist greek speaking
mw2EFWkBP7Y1UR1142_A) health insurance data scientist london
Ow2EFWkBP7Y1UR1183MA) bioprocessing scientist
eg2EFWkBP7Y1UR1163HD) nlp scientist
7w2EFWkBP7Y1UR116XBp) senior research scientist


In [94]:
searchFunctionAllFeatures("hadoop")

52 documents found
dg2EFWkBP7Y1UR1173L0) {'title': 'hadoop data engineer', 'company': 'computer futures', 'description': "working market leading data consultancy helps customers manage multiple data types countless varied sources helping solutions data lake machine learning cyber security work closely data scientists help design methods collecting integrating variety data used predictive analytics data science uses apply big data technologies hadoop spark streams also nosql data management experience large multi-structured data sets work closely customer 's businesses need master 's degree equivalent computer science computer engineering computer applications related field 2 years experience linux hadoop oracle informatica teradata used tools technologies kafka cassandra storm spark streams nosql looking new position think skills needed please apply cv today find computer futures please visit www.computerfutures.com computer futures trading division sthree partnership llp acting employ